In [1]:
from langdetect import detect
from deep_translator import GoogleTranslator

In [2]:
def detect_and_translate(text):
    # Detect the language of the text
    detected_language = detect(text)
    
    if detected_language != 'en':
        # Translate the text to English
        translator = GoogleTranslator(source='auto', target='en')
        translated_text = translator.translate(text)
        return translated_text
    else:
        return text

In [3]:
# Example usage:
job_offer = """
Votre mission sera de développer et de maintenir des applications web en utilisant les dernières technologies.
"""

translated_job_offer = detect_and_translate(job_offer)
print(translated_job_offer)

Original language detected: fr
Translation completed.
Your mission will be to develop and maintain web applications using the latest technologies.
